# Incorporating additional space parameters to solve overfitting
## Various iterations of the proccess of: cross validation->training->prediction on test are done below for various types of space parameters

In [3]:
import sys
!{sys.executable} -m pip install seaborn
import seaborn

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [ ]:
#Max Boosting = 1000 (change from 2k to 1k)

# Function: K Folds Cross Validation using Hyperopt

In [1]:
data_path = "C:/Users/Somasundaran/Sanchita- Sales Prediction/"
submission_path= data_path+'submission/'
fold_path = data_path+'fold_data/'

from Functions import *

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import lightgbm as lgb

%matplotlib inline
seed=1204

cv_loss_list=[]
n_iteration_list=[]

def score(params):
    print("Training with params: ")
    print(params)
    cv_losses=[]
    cv_iteration=[]
    for (train_idx,val_idx) in cv:
        cv_train = X.iloc[train_idx]
        cv_val = X.iloc[val_idx]
        cv_y_train = y[train_idx]
        cv_y_val = y[val_idx]
        lgb_model = lgb.train(params, lgb.Dataset(cv_train, label=cv_y_train), 1000, 
                          lgb.Dataset(cv_val, label=cv_y_val), verbose_eval=False, 
                          early_stopping_rounds=100)
       
        train_pred = lgb_model.predict(cv_train,lgb_model.best_iteration+1)
        val_pred = lgb_model.predict(cv_val,lgb_model.best_iteration+1)
        
        val_loss = root_mean_squared_error(cv_y_val,val_pred)
        train_loss = root_mean_squared_error(cv_y_train,train_pred)
        print('Train RMSE: {}. Val RMSE: {}'.format(train_loss,val_loss))
        print('Best iteration: {}'.format(lgb_model.best_iteration))
        cv_losses.append(val_loss)
        cv_iteration.append(lgb_model.best_iteration)
    print('6 fold results: {}'.format(cv_losses))
    cv_loss_list.append(cv_losses)
    n_iteration_list.append(cv_iteration)
    
    mean_cv_loss = np.mean(cv_losses)
    print('Average iterations: {}'.format(np.mean(cv_iteration)))
    print("Mean Cross Validation RMSE: {}\n".format(mean_cv_loss))
    return {'loss': mean_cv_loss, 'status': STATUS_OK}

def optimize(space,seed=seed,max_evals=5):
    
    best = fmin(score, space, algo=tpe.suggest, 
        # trials=trials, 
        max_evals=max_evals)
    return best

In [2]:
all_data = get_all_data(data_path,'new_sales_lag_after12.pickle')

X,y = get_X_y(all_data,33)
X.drop('date_block_num',axis=1,inplace=True)

cv = get_cv_idxs(all_data,28,33)

## Defining initial parameters

In [8]:
lgb_params = {
               'colsample_bytree': 0.75,
               'metric': 'rmse',
               'min_data_in_leaf': 128, 
               'subsample': 0.75, 
               'learning_rate': 0.05, 
               'objective': 'regression', 
               'bagging_seed': 128, 
               'num_leaves': 128,
               'bagging_freq':1,
               'seed':1204,
    
                'max_depth':10
              }

## Training on full dataset

In [9]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 800, 
                      lgb.Dataset(X, label=y), verbose_eval=10)

[10]	valid_0's rmse: 1.00189
[20]	valid_0's rmse: 0.913893
[30]	valid_0's rmse: 0.871634
[40]	valid_0's rmse: 0.848812
[50]	valid_0's rmse: 0.835027
[60]	valid_0's rmse: 0.826329
[70]	valid_0's rmse: 0.820018
[80]	valid_0's rmse: 0.815058
[90]	valid_0's rmse: 0.810961
[100]	valid_0's rmse: 0.807412
[110]	valid_0's rmse: 0.804104
[120]	valid_0's rmse: 0.800666
[130]	valid_0's rmse: 0.7979
[140]	valid_0's rmse: 0.796012
[150]	valid_0's rmse: 0.793724
[160]	valid_0's rmse: 0.791076
[170]	valid_0's rmse: 0.789228
[180]	valid_0's rmse: 0.787496
[190]	valid_0's rmse: 0.78564
[200]	valid_0's rmse: 0.783352
[210]	valid_0's rmse: 0.781588
[220]	valid_0's rmse: 0.780076
[230]	valid_0's rmse: 0.777793
[240]	valid_0's rmse: 0.776475
[250]	valid_0's rmse: 0.77534
[260]	valid_0's rmse: 0.772513
[270]	valid_0's rmse: 0.771484
[280]	valid_0's rmse: 0.770399
[290]	valid_0's rmse: 0.769121
[300]	valid_0's rmse: 0.767691
[310]	valid_0's rmse: 0.76633
[320]	valid_0's rmse: 0.76559
[330]	valid_0's rmse: 0.

## Predicting on Test Dataset

In [10]:
test_lag = pd.read_csv(os.path.join(data_path, 'test_lag_data.csv'),encoding = "ISO-8859-1")
test_lag.drop(['ID','item_name','date_block_num'],axis=1,inplace=True)
test_lag_pred = lgb_model_full.predict(test_lag,800)

In [12]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ="C:/Users/Somasundaran/Sanchita- Sales Prediction/" ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub
get_submission(test_lag_pred,'lightgbm_basic_6folds- Parameter tuning v1');

In [13]:
## Tweaking the above parameters

In [14]:
lgb_params = {
               'colsample_bytree': 0.75,
               'metric': 'rmse',
               'min_data_in_leaf': 128, 
               'subsample': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'regression', 
               'bagging_seed': 128, 
               'num_leaves': 128,
               'bagging_freq':1,
               'seed':1204
              }

In [15]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 708, 
                      lgb.Dataset(X, label=y), verbose_eval=10)

[10]	valid_0's rmse: 1.05698
[20]	valid_0's rmse: 0.972714
[30]	valid_0's rmse: 0.918783
[40]	valid_0's rmse: 0.882679
[50]	valid_0's rmse: 0.858197
[60]	valid_0's rmse: 0.841073
[70]	valid_0's rmse: 0.828324
[80]	valid_0's rmse: 0.818616
[90]	valid_0's rmse: 0.811198
[100]	valid_0's rmse: 0.805408
[110]	valid_0's rmse: 0.800574
[120]	valid_0's rmse: 0.79646
[130]	valid_0's rmse: 0.792662
[140]	valid_0's rmse: 0.789279
[150]	valid_0's rmse: 0.786108
[160]	valid_0's rmse: 0.783363
[170]	valid_0's rmse: 0.780447
[180]	valid_0's rmse: 0.777746
[190]	valid_0's rmse: 0.775659
[200]	valid_0's rmse: 0.773514
[210]	valid_0's rmse: 0.771435
[220]	valid_0's rmse: 0.769754
[230]	valid_0's rmse: 0.768097
[240]	valid_0's rmse: 0.766674
[250]	valid_0's rmse: 0.765152
[260]	valid_0's rmse: 0.763868
[270]	valid_0's rmse: 0.762353
[280]	valid_0's rmse: 0.760885
[290]	valid_0's rmse: 0.75949
[300]	valid_0's rmse: 0.758241
[310]	valid_0's rmse: 0.757211
[320]	valid_0's rmse: 0.756147
[330]	valid_0's rmse

In [17]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ="C:/Users/Somasundaran/Sanchita- Sales Prediction/" ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub
get_submission(test_lag_pred,'lightgbm_basic_6folds- Parameter tuning v1.1');

## Running hyperopt for different set of space parameters

In [ ]:
# Max depth
# L1 regularization
# L2 regularization

In [3]:
space = {
    'max_depth': hp.choice('max_depth', np.arange(5, 15, dtype=int)),
    'subsample': hp.quniform('subsample', 0.5, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'min_data_in_leaf': hp.choice('min_data_in_leaf',np.arange(20, 150,5, dtype=int)),
    'learning_rate': hp.quniform('learning_rate', 0.025, 0.5, 0.025),
    'seed':seed,
    'objective': 'regression',
    'metric':'rmse',
    
    'lambda_l1': hp.choice('lambda_l1',  [0, 0.0001, 0.01, 0.1, 1, 10]),
    'lambda_l2': hp.choice('lambda_l2',  [0, 0.0001, 0.01, 0.1, 1, 10]),
}
best_hyperparams = optimize(space,max_evals=5)
print("The best hyperparameters are: ")
print(best_hyperparams)

Training with params: 
{'colsample_bytree': 0.65, 'lambda_l1': 0.01, 'lambda_l2': 0.0001, 'learning_rate': 0.2, 'max_depth': 10, 'metric': 'rmse', 'min_data_in_leaf': 95, 'objective': 'regression', 'seed': 1204, 'subsample': 0.55}
Train RMSE: 0.7816968520113593. Val RMSE: 0.8266133049486397
Best iteration: 141
Train RMSE: 0.8489706795981492. Val RMSE: 0.7789064230998629
Best iteration: 23
Train RMSE: 0.7485514005682771. Val RMSE: 0.7096606138852695
Best iteration: 301
Train RMSE: 0.7292370186092529. Val RMSE: 0.7695765839774419
Best iteration: 437
Train RMSE: 0.7676144186998185. Val RMSE: 0.8844194355914482
Best iteration: 176
Train RMSE: 0.8133357031504567. Val RMSE: 0.9341165627161099
Best iteration: 52
6 fold results: [0.8266133049486397, 0.7789064230998629, 0.7096606138852695, 0.7695765839774419, 0.8844194355914482, 0.9341165627161099]
Average iterations: 188.33333333333334
Mean Cross Validation RMSE: 0.8172154873697952

Training with params: 
{'colsample_bytree': 0.600000000000000

In [3]:
lgb_params = {
               'colsample_bytree': 0.6000000000000001,
               'metric': 'rmse',
               'min_data_in_leaf': 3, 
               'subsample': 0.5, 
               'learning_rate': 0.1, 
               'objective': 'regression', 
               'bagging_seed': 128, 
               'num_leaves': 128,
               'bagging_freq':1,
               'seed':1204,
                'max_depth': 9,
                'lambda_l1': 5, 
                'lambda_l2': 3
    
              }

In [4]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 800, 
                      lgb.Dataset(X, label=y), verbose_eval=100)

[100]	valid_0's rmse: 0.784286
[200]	valid_0's rmse: 0.75534
[300]	valid_0's rmse: 0.737224
[400]	valid_0's rmse: 0.722336
[500]	valid_0's rmse: 0.709543
[600]	valid_0's rmse: 0.698375
[700]	valid_0's rmse: 0.690354
[800]	valid_0's rmse: 0.681439
Wall time: 12min 16s


In [7]:
test_lag = pd.read_csv(os.path.join(data_path, 'test_lag_data.csv'), encoding = "ISO-8859-1")
test_lag.drop(['ID','item_name','date_block_num'],axis=1,inplace=True)

In [8]:
test_lag_pred = lgb_model_full.predict(test_lag,800)

In [9]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ="C:/Users/Somasundaran/Sanchita- Sales Prediction/" ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub
get_submission(test_lag_pred,'lightgbm_basic_6folds- Parameter tuning v2');

## Running on revised test set (Including holiday features)

In [3]:
lgb_params = {
               'colsample_bytree': 0.6000000000000001,
               'metric': 'rmse',
               'min_data_in_leaf': 3, 
               'subsample': 0.5, 
               'learning_rate': 0.1, 
               'objective': 'regression', 
               'bagging_seed': 128, 
               'num_leaves': 128,
               'bagging_freq':1,
               'seed':1204,
                'max_depth': 9,
                'lambda_l1': 5, 
                'lambda_l2': 3
    
              }

In [4]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 800, 
                      lgb.Dataset(X, label=y), verbose_eval=100)

[100]	valid_0's rmse: 0.784286
[200]	valid_0's rmse: 0.75534
[300]	valid_0's rmse: 0.737224
[400]	valid_0's rmse: 0.722336
[500]	valid_0's rmse: 0.709543
[600]	valid_0's rmse: 0.698375
[700]	valid_0's rmse: 0.690354
[800]	valid_0's rmse: 0.681439
Wall time: 12min 39s


In [5]:
test_lag = pd.read_csv(os.path.join(data_path, 'test_lag_data_Holiday_features.csv'), encoding = "ISO-8859-1")
test_lag.drop(['ID','item_name','date_block_num'],axis=1,inplace=True)

In [6]:
test_lag.head()

,item_block_target_mean,item_block_target_sum,item_cat_block_target_mean,item_cat_block_target_sum,item_category_id,item_id,shop_block_target_mean,shop_block_target_sum,shop_id,target,...,shop_block_target_mean_lag_12,shop_block_target_sum_lag_12,item_cat_block_target_mean_lag_12,item_cat_block_target_sum_lag_12,target_lag_12,Newyear_Xmas,Valentine_MenDay,WomenDay,Easter_Labor,December
0,0.0,0.0,0.0,0.0,19,5037,0.0,0.0,5,0.0,...,1.235043,1445.0,1.144403,6134.0,1.0,0,0,0,0,0
1,0.0,0.0,0.0,0.0,55,5320,0.0,0.0,5,0.0,...,1.235043,1445.0,1.041406,9809.0,0.0,0,0,0,0,0
2,0.0,0.0,0.0,0.0,19,5233,0.0,0.0,5,0.0,...,1.235043,1445.0,1.144403,6134.0,0.0,0,0,0,0,0
3,0.0,0.0,0.0,0.0,23,5232,0.0,0.0,5,0.0,...,1.235043,1445.0,1.099875,5275.0,0.0,0,0,0,0,0
4,0.0,0.0,0.0,0.0,20,5268,0.0,0.0,5,0.0,...,1.235043,1445.0,2.338131,12834.0,0.0,0,0,0,0,0


In [7]:
test_lag_pred = lgb_model_full.predict(test_lag,800)

In [8]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ="C:/Users/Somasundaran/Sanchita- Sales Prediction/" ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub
get_submission(test_lag_pred,'lightgbm_basic_6folds-Revised Test-Parameter tuning v1');

# Modifying Space Parameters- to combat overfitting

In [10]:
space = {
    'max_depth': hp.choice('max_depth', np.arange(5, 25, dtype=int)),
    'subsample': hp.quniform('subsample', 0.6, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 1, 0.05),
    'min_data_in_leaf': hp.choice('min_data_in_leaf',np.arange(50, 250,5, dtype=int)),
    'learning_rate': hp.quniform('learning_rate', 0.025, 0.5, 0.025),
    'seed':seed,
    'objective': 'regression',
    'metric':'rmse',
    
    'lambda_l1': hp.choice('lambda_l1',  [0, 0.0001, 0.01, 0.1, 1]),
    'lambda_l2': hp.choice('lambda_l2',  [0, 0.0001, 0.01, 0.1, 1]),
}
best_hyperparams = optimize(space,max_evals=10)
print("The best hyperparameters are: ")
print(best_hyperparams)

Training with params: 
{'colsample_bytree': 0.8, 'lambda_l1': 0.01, 'lambda_l2': 0, 'learning_rate': 0.325, 'max_depth': 23, 'metric': 'rmse', 'min_data_in_leaf': 130, 'objective': 'regression', 'seed': 1204, 'subsample': 0.9500000000000001}
Train RMSE: 0.7921803749105867. Val RMSE: 0.8326647375713375
Best iteration: 67
Train RMSE: 0.7960730457120009. Val RMSE: 0.7654023603541752
Best iteration: 62
Train RMSE: 0.7386634079310335. Val RMSE: 0.710497573063942
Best iteration: 236
Train RMSE: 0.7604421244637175. Val RMSE: 0.7775750235172315
Best iteration: 139
Train RMSE: 0.7798507879899743. Val RMSE: 0.8851037539988987
Best iteration: 78
Train RMSE: 0.7875323848974793. Val RMSE: 0.9169401947469903
Best iteration: 65
6 fold results: [0.8326647375713375, 0.7654023603541752, 0.710497573063942, 0.7775750235172315, 0.8851037539988987, 0.9169401947469903]
Average iterations: 107.83333333333333
Mean Cross Validation RMSE: 0.8146972738754292

Training with params: 
{'colsample_bytree': 0.9, 'lamb

Train RMSE: 0.8039210640921063. Val RMSE: 0.886895916387914
Best iteration: 63
Train RMSE: 0.8542421990735928. Val RMSE: 0.9257487061867942
Best iteration: 14
6 fold results: [0.8317319885200577, 0.7773149387684525, 0.7112955539544094, 0.7722645478990114, 0.886895916387914, 0.9257487061867942]
Average iterations: 155.33333333333334
Mean Cross Validation RMSE: 0.8175419419527731

Training with params: 
{'colsample_bytree': 0.8, 'lambda_l1': 0.1, 'lambda_l2': 1, 'learning_rate': 0.125, 'max_depth': 21, 'metric': 'rmse', 'min_data_in_leaf': 110, 'objective': 'regression', 'seed': 1204, 'subsample': 0.7000000000000001}
Train RMSE: 0.7610631480930864. Val RMSE: 0.832591594481484
Best iteration: 322
Train RMSE: 0.7630128558843561. Val RMSE: 0.7669208172631089
Best iteration: 342
Train RMSE: 0.7053852129517231. Val RMSE: 0.6971955198715297
Best iteration: 993
Train RMSE: 0.7247210603361951. Val RMSE: 0.7680042262716101
Best iteration: 700
Train RMSE: 0.7611515914659712. Val RMSE: 0.8890055265

In [12]:
lgb_params = {
               'colsample_bytree': 0.65,
               'metric': 'rmse',
               'min_data_in_leaf': 36, 
               'subsample': 0.7000000000000001, 
               'learning_rate': 0.05, 
               'objective': 'regression', 
               'bagging_seed': 128, 
    
               'num_leaves': 128,
    
               'bagging_freq':1,
               'seed':1204,
                'max_depth': 12,
                'lambda_l1': 3, 
                'lambda_l2': 1
    
              }

In [13]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 1000, 
                      lgb.Dataset(X, label=y), verbose_eval=100)

[100]	valid_0's rmse: 0.791158
[200]	valid_0's rmse: 0.762947
[300]	valid_0's rmse: 0.745712
[400]	valid_0's rmse: 0.733524
[500]	valid_0's rmse: 0.72312
[600]	valid_0's rmse: 0.714322
[700]	valid_0's rmse: 0.704718
[800]	valid_0's rmse: 0.698059
[900]	valid_0's rmse: 0.691788
[1000]	valid_0's rmse: 0.68611
Wall time: 13min 50s
Parser   : 207 ms


In [14]:
test_lag_pred = lgb_model_full.predict(test_lag)

In [15]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ="C:/Users/Somasundaran/Sanchita- Sales Prediction/" ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub
get_submission(test_lag_pred,'lightgbm_basic_6folds- Parameter tuning v3');

## Tweaking the above parameters

In [16]:
lgb_params = {
               'colsample_bytree': 0.65,
               'metric': 'rmse',
               'min_data_in_leaf': 500, 
               'subsample': 0.7000000000000001, 
               'learning_rate': 0.05, 
               'objective': 'regression', 
               'bagging_seed': 128, 
    
               'num_leaves': 256,
    
               'bagging_freq':1,
               'seed':1204,
                'max_depth': 12,
                'lambda_l1': 3, 
                'lambda_l2': 1
    
              }

In [17]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 1000, 
                      lgb.Dataset(X, label=y), verbose_eval=100)

[100]	valid_0's rmse: 0.805414
[200]	valid_0's rmse: 0.791468
[300]	valid_0's rmse: 0.78076
[400]	valid_0's rmse: 0.772698
[500]	valid_0's rmse: 0.766313
[600]	valid_0's rmse: 0.761188
[700]	valid_0's rmse: 0.756788
[800]	valid_0's rmse: 0.753099
[900]	valid_0's rmse: 0.749767
[1000]	valid_0's rmse: 0.746465
Wall time: 22min 33s


In [18]:
test_lag_pred = lgb_model_full.predict(test_lag)

In [19]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ="C:/Users/Somasundaran/Sanchita- Sales Prediction/" ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub
get_submission(test_lag_pred,'lightgbm_basic_6folds- Parameter tuning v3.1');

# Incorporating maximum number of leaves + Change in space parameters

In [4]:
space = {
    'max_depth': hp.choice('max_depth', np.arange(5, 10, dtype=int)),
    'subsample': hp.quniform('subsample', 0.6, 1, 0.05),
    'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 1, 0.05),
    'min_data_in_leaf': hp.choice('min_data_in_leaf',np.arange(50, 500,5, dtype=int)),
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.1, 0.01),
    'seed':seed,
    'objective': 'regression',
    'metric':'rmse',
    
    'lambda_l1': hp.choice('lambda_l1',  [0, 0.0001, 0.01, 0.1, 1]),
    'lambda_l2': hp.choice('lambda_l2',  [0, 0.0001, 0.01, 0.1, 1]),
    
    'num_leaves': hp.choice('num_leaves',np.arange(32, 129,32, dtype=int))
}
best_hyperparams = optimize(space,max_evals=3)
print("The best hyperparameters are: ")
print(best_hyperparams)

Training with params: 
{'colsample_bytree': 0.9, 'lambda_l1': 0, 'lambda_l2': 0.1, 'learning_rate': 0.07, 'max_depth': 9, 'metric': 'rmse', 'min_data_in_leaf': 80, 'num_leaves': 32, 'objective': 'regression', 'seed': 1204, 'subsample': 0.9}
Train RMSE: 0.759974846122296. Val RMSE: 0.8334317219916333
Best iteration: 611
Train RMSE: 0.8237353103591266. Val RMSE: 0.7754100590561498
Best iteration: 133
Train RMSE: 0.735915092319946. Val RMSE: 0.7024875837325822
Best iteration: 991
Train RMSE: 0.7363734525472706. Val RMSE: 0.7633673688177264
Best iteration: 945
Train RMSE: 0.8068807480595299. Val RMSE: 0.8864482861470799
Best iteration: 169
Train RMSE: 0.7817603803440171. Val RMSE: 0.9056419639185334
Best iteration: 363
6 fold results: [0.8334317219916333, 0.7754100590561498, 0.7024875837325822, 0.7633673688177264, 0.8864482861470799, 0.9056419639185334]
Average iterations: 535.3333333333334
Mean Cross Validation RMSE: 0.8111311639439508

Training with params: 
{'colsample_bytree': 0.850000

In [5]:
# 'colsample_bytree': 0.9, 'lambda_l1': 0, 'lambda_l2': 0.1, 'learning_rate': 0.07, 'max_depth': 9, 'metric': 'rmse', 
# 'min_data_in_leaf': 80, 'num_leaves': 32, 'objective': 'regression', 'seed': 1204, 'subsample': 0.9
                    
lgb_params = {
               'colsample_bytree': 0.9,
               'metric': 'rmse',
               'min_data_in_leaf': 80, 
               'subsample': 0.9, 
               'learning_rate': 0.07, 
               'objective': 'regression', 
               'bagging_seed': 128, 
               'num_leaves': 128,
               'bagging_freq':1,
               'seed':1204,
                'max_depth': 9,
                'lambda_l1': 0, 
                'lambda_l2': 0.1
    
              }                    

In [6]:
%%time
lgb_model_full = lgb.train(lgb_params, lgb.Dataset(X, label=y), 800, 
                      lgb.Dataset(X, label=y), verbose_eval=100)

[100]	valid_0's rmse: 0.796542
[200]	valid_0's rmse: 0.768223
[300]	valid_0's rmse: 0.748677
[400]	valid_0's rmse: 0.735852
[500]	valid_0's rmse: 0.724154
[600]	valid_0's rmse: 0.715332
[700]	valid_0's rmse: 0.708109
[800]	valid_0's rmse: 0.701862
Wall time: 16min 8s


In [9]:
test_lag = pd.read_csv(os.path.join(data_path, 'test_lag_data.csv'), encoding = "ISO-8859-1")
test_lag.drop(['ID','item_name','date_block_num'],axis=1,inplace=True)
test_lag_pred = lgb_model_full.predict(test_lag)

In [10]:
def get_submission(item_cnt_month,sub_name,clip=20,data_path ="C:/Users/Somasundaran/Sanchita- Sales Prediction/" ):
    item_cnt_month = np.clip(item_cnt_month,0,clip)
    test= pd.read_csv(os.path.join(data_path, 'test.csv.gz'))
    sub = test.copy()
    sub['item_cnt_month'] = item_cnt_month
    sub.drop(['item_id','shop_id'],axis=1,inplace=True)
    sub.to_csv(data_path+'submission/' + sub_name+'.csv',index=False)
    return sub
get_submission(test_lag_pred,'lightgbm_basic_6folds- Parameter tuning v4');